In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd
import sys
import os
sys.path.append(os.path.abspath('../'))
import pandas as pd
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.conllu_reader import CoNNLUReader

/Users/rabauti/repos/eki/prg1978__opikulausetekorpus/heete_loendid


# Tabel verb + da andmete kogumine


**Ülesande püstitus**
    
Kõik laused, kus esineb tabelis [list_da.csv](list_da.csv) olev verb ja verbil on otsene alluv deprel=xcomp, feats sisaldab inf

**Tulemus** 

Tabel veergudega:
1. leitud lause, 
2. milline tabelis olevates verbidest seal esineb (algvormis), 
3. da-infinitiivi vormis oleva verbi lemma, 
4. keeletase.
   

In [3]:
%%time

# verbid etteantud nimekirjast
df_verbs = pd.read_csv('list_da.csv')
my_verbs = list(df_verbs['lemma'].unique())


file_name = '../data/Model2Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu'
my_reader = CoNNLUReader(
   file_name = file_name
)


CPU times: user 1.88 ms, sys: 1.87 ms, total: 3.76 ms
Wall time: 6.55 ms


In [4]:
my_verbs

['saama',
 'suutma',
 'jaksama',
 'jõudma',
 'nägema',
 'oskama',
 'teadma',
 'mõistma',
 'tohtima',
 'võima',
 'tahtma',
 'kavatsema',
 'plaanima',
 'otsustama',
 'lootma',
 'soovima',
 'igatsema',
 'ihkama',
 'maldama',
 'kärsima',
 'läbema',
 'unistama',
 'ootama',
 'himustama',
 'taotlema',
 'ilgema',
 'sügelema',
 'kibelema',
 'janunema',
 'kaaluma',
 'kavandama',
 'kokku leppima',
 'mõtlema',
 'plaanitsema',
 'planeerima',
 'sihtima',
 'märkama',
 'taipama',
 'unustama',
 'kartma',
 'häbenema',
 'armastama',
 'eelistama',
 'julgema',
 'söandama',
 'tihkama',
 'riskima',
 'usaldama',
 'riskeerima',
 'uskuma',
 'suvatsema',
 'viitsima',
 'paljuks pidama',
 'raatsima',
 'täima',
 'vaevaks võtma',
 'pelgama',
 'põlgama',
 'tõrkuma',
 'kõhklema',
 'pruukima',
 'tarvitsema',
 'lubama',
 'ähvardama',
 'tõotama',
 'vanduma',
 'proovima',
 'püüdma',
 'katsuma',
 'üritama',
 'tavatsema',
 'harrastama',
 'väärima']

In [5]:
%%time

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")

collected_data = []
count = 0
for collection_id, graph in my_reader.get_sentences():
    # matrix for node distances
    dpath = graph.get_distances_matrix()
    
    # verb nodes
    verb_nodes = [v for v in graph.get_nodes_by_attributes(attrname="POS", attrvalue="VERB") if graph.nodes[v]["lemma"] in my_verbs]
    if not len(verb_nodes): continue
    
    # xcomp
    xcomp_nodes = graph.get_nodes_by_attributes(attrname="deprel", attrvalue="xcomp")
   
    if not len(xcomp_nodes): continue
    
    for verb in verb_nodes:
        # childnodes
        kids = [k for k in dpath[verb] if dpath[verb][k] == 1]
        for xcomp in xcomp_nodes:
            if xcomp not in kids:
                continue
            if not graph.nodes[xcomp]["feats"] or "VerbForm" not in graph.nodes[xcomp]["feats"].keys() or not graph.nodes[xcomp]["feats"]["VerbForm"] == 'Inf':
                continue
            
            #graph.draw_graph2(highlight=[verb, xcomp])
            d = {
                'id':  graph.get_metadata('row_id'),
                'sentence':  graph.get_metadata('text'),
                'verb':  graph.nodes[verb]["lemma"],
                'xcomp':  graph.nodes[xcomp]["lemma"],
                'level':  graph.get_metadata('sent_level'),
                'sub': " ".join(
                            [graph.nodes[n]["form"] for n in sorted([verb] + kids)]
                        ),
            }
            
            collected_data.append(d)


../data/Eesti-keele-kui-teise-keele-kooliõpikute-lausete-korpus-2021.conllu
CPU times: user 3.75 s, sys: 58.4 ms, total: 3.81 s
Wall time: 3.86 s


In [7]:
df = pd.DataFrame.from_dict(collected_data)
df.to_csv(f'verb_xcompInf_{date_time}.csv', index=None)
df.head(30)

,id,sentence,verb,xcomp,level,sub
0,5,"Nendel põhikooli lõpetajatel, kes on otsustanu...",otsustama,jätkama,gümn,", kes on otsustanud jätkata ,"
1,9,"Üha rohkem leidub noori, kes pärast gümnaasium...",soovima,astuma,gümn,", kes lõpetamist soovivad astuda"
2,36,Isegi pastakat ei julgenud kelleltki küsida.,julgema,küsima,gümn,pastakat ei julgenud küsida .
3,37,"Ükskord püüdsin juttu alustada nii, et küsisin...",püüdma,alustama,gümn,Ükskord püüdsin alustada küsisin .
4,57,"Kui lähed uude kooli, püüa leida vähemalt üks ...",püüdma,leidma,gümn,lähed püüa leida .
5,116,Kui tahad tunni ajal minna WC-sse või arsti ju...,tahtma,minema,gümn,"Kui tahad minna ,"
6,128,"Kui sa tahad tunni ajal näiteks WC-sse minna, ...",tahtma,minema,gümn,"Kui sa tahad minna ,"
7,146,"Koolis tohtis end meikida, kuid kõrvarõngaid e...",tohtima,meikima,gümn,Koolis tohtis meikida kõrvarõngaid .
8,209,"Nad oskavad tekitada vaidlust ka siis, kui esm...",oskama,tekitama,gümn,Nad oskavad tekitada siis vaielda .
9,247,Kolmandik vene koolinoortest tahaks õppida Tar...,tahtma,õppima,gümn,Kolmandik tahaks õppida .
